In [1]:
from functools import partial
import jax
import jax.numpy as jnp

In [2]:
!nvidia-smi

Thu Feb  9 10:50:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
@jax.jit
def f(x):
    return 2 * x * x * jnp.cos(x) - 5 * x

In [4]:
@jax.jit
def df(x):
    return 4 * x * jnp.cos(x) - 2 * x * x * jnp.sin(x) - 5

In [5]:
@partial(jax.jit, static_argnames=['alpha', 'epochs'])
def gradient_descent(x, y, alpha: float, epochs: int):
    for i in range(epochs):
        x = jnp.concatenate([x, jnp.array([x[i] - alpha * df(x[i])])])
        y = jnp.concatenate([y, jnp.array([f(x[i + 1])])])
    return x, y

In [6]:
curve = jnp.linspace(-5, 5, 100)
alpha = 0.05
epochs = 20
x = jnp.array([-1.])
y = jnp.array([f(x[0])])
%timeit gradient_descent(x=x, y=y, alpha=alpha, epochs=epochs)

67.1 µs ± 27.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
